In [17]:
import pandas as pd
from jira import JIRA
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os

# --- Load JIRA Configuration from Settings ---
from notebook_helper import load_jira_config, get_project_list, validate_setup

# Validate that everything is set up correctly
if not validate_setup():
    raise ValueError("❌ Notebook setup incomplete. Please configure settings in the main dashboard.")

# Load JIRA configuration from settings system
jira_config = load_jira_config()
JIRA_URL = jira_config.get('jira_url', '')
EMAIL = jira_config.get('username', '')
API_TOKEN = jira_config.get('api_token', '')

# Project configuration - you can customize this
PROJECT_KEY = 'IOS'  # Change this to your target project
Environment = 'Production'  # Production or Staging

print(f"\n📂 Target Project: {PROJECT_KEY}")
print(f"🌍 Environment: {Environment}")
print("🚀 Ready to run bug risk analysis!") 


# --- Connect to JIRA --- 
jira = JIRA(server=JIRA_URL, basic_auth=(EMAIL, API_TOKEN))

# --- Fetch Issues (Bugs and Support Tickets) ---
jql = (
    f'project = "{PROJECT_KEY}" AND '
    f'(type = "Bug" OR type = "Support Ticket")'
)
issues = jira.search_issues(jql, maxResults=2000, fields="summary,status,created,resolutiondate,assignee,reporter,priority,environment,components")

# --- Collect Data ---
data = []
for issue in issues:
    env = getattr(issue.fields, 'Environment', None)
    print(f"Issue {issue.key} Environment: {env}")  # Debug print
    # Only include issues where environment contains 'Production' (case-insensitive)
    if env and 'production' in str(env).lower():
        data.append({
            'key': issue.key,
            'summary': issue.fields.summary,
            'status': issue.fields.status.name,
            'Created': issue.fields.created,
            'resolved': getattr(issue.fields.resolutiondate, 'isoformat', lambda: None)(),
            'assignee': getattr(issue.fields.assignee, 'displayName', None),
            'reporter': getattr(issue.fields.reporter, 'displayName', None),
            'priority': getattr(issue.fields.priority, 'name', None),
            'environment': env,
            'Components': ', '.join([c.name for c in issue.fields.components]) if hasattr(issue.fields, 'components') else None,
        })
df = pd.DataFrame(data)

if not df.empty:
    df.to_csv('../data/processed/jira-ANDst-bugs.csv', index=False)
    data = pd.read_csv('../data/processed/jira-ANDst-bugs.csv')
else:
    data = df  # keep as empty DataFrame

print(data.columns)

if not df.empty and 'priority' in df.columns:
    df['priority'] = df['priority'].fillna('Unknown')
    df.to_csv('jira_production_bugs.csv', index=False)
    print(f"Fetched {len(df)} issues for Production environment.")
    print(df.columns)
    print(df.head())
else:
    print("No issues found with 'Production' in the environment field.")
    print("Check the printed environments above to adjust your filter if needed.")

# --- Predicting High-Risk Bugs with Machine Learning ---

if not df.empty and 'priority' in df.columns:
    # Example: Define high risk as priority 'Highest' or 'High'
    df['High_Risk'] = df['priority'].apply(lambda x: 1 if str(x).lower() in ['highest', 'high', 'blocker', 'critical'] else 0)
    # Feature selection (drop non-features)
    drop_cols = ['key', 'summary', 'Created', 'resolved', 'assignee', 'reporter', 'environment', 'High_Risk']
    X = df.drop(columns=[col for col in drop_cols if col in df.columns])
    y = df['High_Risk']

    # Encode categorical features
    X = pd.get_dummies(X, drop_first=True)

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Train model
    clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    clf.fit(X_train, y_train)

    # Evaluate
    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred))

    # Feature importance
    import matplotlib.pyplot as plt
    import seaborn as sns

    importances = clf.feature_importances_
    feature_names = X_train.columns
    feat_imp = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
    feat_imp = feat_imp[feat_imp['Importance'] > 0].sort_values(by='Importance', ascending=False).head(15)

    plt.figure(figsize=(8, 6))
    sns.barplot(x='Importance', y='Feature', data=feat_imp)
    plt.title('Top XGBoost Feature Importances (Production Bugs)')
    plt.tight_layout()
    plt.show()
else:
    print("Skipping ML steps: No data available with 'priority' column.")

Issue IOS-5722 Environment: None
Issue IOS-5718 Environment: None
Issue IOS-5717 Environment: None
Issue IOS-5716 Environment: None
Issue IOS-5715 Environment: None
Issue IOS-5695 Environment: None
Issue IOS-5694 Environment: None
Issue IOS-5682 Environment: None
Issue IOS-5681 Environment: None
Issue IOS-5680 Environment: None
Issue IOS-5679 Environment: None
Issue IOS-5675 Environment: None
Issue IOS-5673 Environment: None
Issue IOS-5672 Environment: None
Issue IOS-5671 Environment: None
Issue IOS-5670 Environment: None
Issue IOS-5661 Environment: None
Issue IOS-5660 Environment: None
Issue IOS-5659 Environment: None
Issue IOS-5658 Environment: None
Issue IOS-5656 Environment: None
Issue IOS-5652 Environment: None
Issue IOS-5651 Environment: None
Issue IOS-5648 Environment: None
Issue IOS-5646 Environment: None
Issue IOS-5645 Environment: None
Issue IOS-5640 Environment: None
Issue IOS-5639 Environment: None
Issue IOS-5636 Environment: None
Issue IOS-5633 Environment: None
Issue IOS-